In [1]:
from PyQt5 import sip
from PyQt5 import QtWidgets,uic,QtGui
from PyQt5.QtWidgets import QLabel
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import cv2
from PIL import Image

In [2]:
def make_prediction(img):

    input_img = np.expand_dims(img, axis=0)

    res = (model.predict(input_img) > 0.5).astype("int32")
    return res

def show_result(img):

    image = cv2.imread(img)

    img = Image.fromarray(image)

    img = img.resize((64,64))

    img = np.array(img)

    pred = make_prediction(img)
    return pred
model = keras.models.load_model('BrainTumorDetection.h5')
def auto_canny(image,sigma = 0.33):
    v = np.median(image)
    lower = int(max(0,(1.0-sigma)*v))
    upper = int(max(255,(1.0-sigma)*v))
    edged = cv2.Canny(image,lower,upper)
    return edged

In [ ]:
def calculate1():
    #reading image
    img_name=call.lineEdit_4.text()
    print(img_name)
    image = cv2.imread(img_name)
    image = cv2.resize(image,(336,264))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    pixmap=QtGui.QPixmap(img_name)
    call.label_2.setPixmap(pixmap)
def calculate2():
    #predict
    img_name=call.lineEdit_4.text()
    prediction = show_result(img_name)
    result = prediction[0][0]
    if result == 0:
        R = 'No Tumor'
        call.lineEdit_3.setText(R)
    if result ==1:
        R = 'Tumor Detected'
        call.lineEdit_3.setText(R)

def calculate3():
    img_name=call.lineEdit_4.text()
    image = cv2.imread(img_name)
    image = cv2.resize(image,(500,590))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY,0.7)
    #cv2_imshow(gray)
    (T, thresh) = cv2.threshold(gray,150,255,cv2.THRESH_BINARY)
    (T, threshInv) = cv2.threshold(gray,150,255,cv2.THRESH_BINARY_INV)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(10,5))
    closed = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel)
    #cv2_imshow(closed)
    closed = cv2.erode(closed,None, iterations = 14)
    closed = cv2.dilate(closed,None, iterations = 12)
    #cv2_imshow(closed)
    ret,mask = cv2.threshold(closed,155,255,cv2.THRESH_BINARY)
    final = cv2.bitwise_and(image,image,mask = mask)
    #cv2_imshow(final)
    canny = auto_canny(closed)
    #cv2_imshow(canny)
    cnts,_ = cv2.findContours(canny.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    z = cv2.drawContours(image,cnts,-1,(255,0,0),thickness=cv2.FILLED)
    filename = 'area.jpg'
    cv2.imwrite(filename, z) 
    pixmap1=QtGui.QPixmap(filename)
    call.label_3.setPixmap(pixmap1)
def calculate4():
    img_name=call.lineEdit_4.text()
    image = cv2.imread(img_name)
    image = cv2.resize(image,(500,590))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY,0.7)
    #cv2_imshow(gray)
    (T, thresh) = cv2.threshold(gray,150,255,cv2.THRESH_BINARY)
    (T, threshInv) = cv2.threshold(gray,150,255,cv2.THRESH_BINARY_INV)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(10,5))
    closed = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel)
    #cv2_imshow(closed)
    closed = cv2.erode(closed,None, iterations = 14)
    closed = cv2.dilate(closed,None, iterations = 12)
    #cv2_imshow(closed)
    ret,mask = cv2.threshold(closed,155,255,cv2.THRESH_BINARY)
    final = cv2.bitwise_and(image,image,mask = mask)
    #cv2_imshow(final)
    canny = auto_canny(closed)
    #cv2_imshow(canny)
    cnts,_ = cv2.findContours(canny.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    pixel_to_mm = 0.01  
    aa =[]
    image_width_mm = image.shape[1] * pixel_to_mm
    image_height_mm = image.shape[0] * pixel_to_mm

    for contour in cnts:
        area_pixels = cv2.contourArea(contour)
        area_mm = area_pixels * (pixel_to_mm ** 2)
        aa.append(area_mm)
        area = max(aa)
        print("Contour area (mm^2):", round(area_mm,2))
    if len(aa) == 0:
        area = 'No tumor'
        call.lineEdit_2.setText(area)
    else:
        
        area = max(aa)
        call.lineEdit_2.setText(str(area))
    print(area)
    #call.lineEdit_2.setText(str(area))
def calculate5():
    pixmap2=QtGui.QPixmap('reset.png')
    call.label_2.setPixmap(pixmap2)
    call.label_3.setPixmap(pixmap2)
    call.lineEdit_3.setText('')
    call.lineEdit_2.setText('')
    call.lineEdit_4.setText('')
###########################
app=QtWidgets.QApplication([])
call=uic.loadUi("interface.ui")
call.pushButton_5.clicked.connect(calculate1)
call.pushButton_2.clicked.connect(calculate2)
call.pushButton_3.clicked.connect(calculate3)
call.pushButton_4.clicked.connect(calculate4)
call.pushButton_6.clicked.connect(calculate5)
call.show()
app.exec()

image2.jpg
1/1 [==============================] - 1s 749ms/step
Contour area (mm^2): 2.6
2.6049
